# Schema Usage

The schema does not actually store any data.  Instead, it is an interface which allows us to interact with numpy/torch tensors in a semantic manner.  It lets us convert between storage vectors (i.e. how we store the building parameters numerically on disk), simulation objects (e.g. Archetypal Templates and PyUmi Shoeboxes) and machine learning model imports (i.e. torch tensors with full hourly schedule data).

## Notebook setup

We need some jank to get relative imports working.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np

## Initialize the Schema

In [2]:
from schema import Schema, ShoeboxGeometryParameter
schema = Schema()

Let's see what's in the schema:

In [3]:
schema.parameter_names

['batch_id',
 'variation_id',
 'base_template',
 'base_epw',
 'width',
 'height',
 'facade_2_footprint',
 'perim_2_footprint',
 'roof_2_footprint',
 'footprint_2_ground',
 'shading_fact',
 'wwr_n',
 'wwr_e',
 'wwr_s',
 'wwr_w',
 'orientation',
 'LightingPowerDensity',
 'EquipmentPowerDensity',
 'PeopleDensity',
 'FacadeRValue',
 'RoofRValue',
 'PartitionRValue',
 'SlabRValue',
 'schedules_seed',
 'schedules']

We can access a schema parameter from the schema with list indexing:

In [4]:
print(schema["width"])
print(schema["schedules"])
print(schema["orientation"])

---width---
shape_storage=(1,), shape_ml=(1,), dtype=scalar
Width [m]
---schedules---
shape_storage=(8, 16), shape_ml=(8, 8760), dtype=matrix
A matrix in the storage vector with operations to apply to schedules; a matrix of timeseries in ml vector
---orientation---
shape_storage=(1,), shape_ml=(4,), dtype=onehot
Shoebox Orientation


We see that each parameter may have multiple different lengths in the storage vector and ML vector.

We can also print a summary of the whole schema:

In [5]:
print(schema)

-------- Schema --------
---- batch_id ----
shape storage: (1,) / shape ml: (0,)
location storage: 0->1 / location ml: 0->0

---- variation_id ----
shape storage: (1,) / shape ml: (0,)
location storage: 1->2 / location ml: 0->0

---- base_template ----
shape storage: (1,) / shape ml: (0,)
location storage: 2->3 / location ml: 0->0

---- base_epw ----
shape storage: (1,) / shape ml: (0,)
location storage: 3->4 / location ml: 0->0

---- width ----
shape storage: (1,) / shape ml: (1,)
location storage: 4->5 / location ml: 0->1

---- height ----
shape storage: (1,) / shape ml: (1,)
location storage: 5->6 / location ml: 1->2

---- facade_2_footprint ----
shape storage: (1,) / shape ml: (1,)
location storage: 6->7 / location ml: 2->3

---- perim_2_footprint ----
shape storage: (1,) / shape ml: (1,)
location storage: 7->8 / location ml: 3->4

---- roof_2_footprint ----
shape storage: (1,) / shape ml: (1,)
location storage: 8->9 / location ml: 4->5

---- footprint_2_ground ----
shape storage: 

We see that the length of the storage vector is significantly smaller than the length the vector the ML model will see.

## Generating new design vectors in storage space

First let's generate a new, empty design vector, and update the Roof R-Value, and then check that it updated correctly:

In [6]:
storage_vector = schema.generate_empty_storage_vector()
schema.update_storage_vector(storage_vector=storage_vector, parameter="RoofRValue", value=25)
schema["RoofRValue"].extract_storage_values(storage_vector)

25.0

If we print out the full vector, we should be able to see th 25 and a whole bunch of zeros:

In [7]:
print(storage_vector)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. 25.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]


Let's create a new batch of designs:

In [8]:
batch_size = 20
storage_batch = schema.generate_empty_storage_batch(batch_size)
storage_batch.shape

(20, 152)

Great, we see that it has 20 design vectors with 151 values each.

Let's try updating all of the facade R-values values in a batch with the same value:

In [9]:
schema.update_storage_batch(storage_batch, parameter="FacadeRValue", value=14)
schema["FacadeRValue"].extract_storage_values_batch(storage_batch)

array([[14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.],
       [14.]])

Now let's try updating an entire batch with random values.  We can also unnormalize the uniform random variable into the desired range:

In [10]:
parameter = "SlabRValue"
n = batch_size
shape = (n, *schema[parameter].shape_storage)
values = np.random.rand(*shape) # create a random sample with appropriate shape
values = schema[parameter].unnormalize(values) # schema parameter must be a numeric type with min/max defined for unnormalize to work
schema.update_storage_batch(storage_batch, parameter=parameter, value=values)
schema[parameter].extract_storage_values_batch(storage_batch)

array([[ 2.7336393 ],
       [ 4.42821877],
       [39.05815075],
       [13.73105396],
       [46.39368248],
       [37.60366721],
       [33.51669072],
       [27.07568715],
       [24.25549448],
       [47.13754119],
       [38.30644038],
       [45.11128711],
       [15.9638463 ],
       [26.25078644],
       [39.10999201],
       [40.16546848],
       [ 5.04684868],
       [ 7.04249498],
       [ 1.3596638 ],
       [44.77629587]])

Finally, let's try updating just a subset of the batch by using the `index` parameter:

*nb: we can also use an int instead of a tuple for `index` to only update a single vector's parameter*

In [11]:
start = 2
n = 8
end = start + n
parameter = "PartitionRValue"
shape = (n, *schema[parameter].shape_storage)
values = np.random.rand(*shape) # create a random sample with appropriate shape

schema.update_storage_batch(storage_batch, index=(start,end), parameter=parameter, value=values)
schema[parameter].extract_storage_values_batch(storage_batch) 

array([[0.        ],
       [0.        ],
       [0.68946438],
       [0.1180692 ],
       [0.45982586],
       [0.19647618],
       [0.96267717],
       [0.73718392],
       [0.84350581],
       [0.65299646],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ]])

A useful technique will be to start with a small batch, and then duplicate it in concatenations along `axis=0` as we build up our mixed grid/hypercube/random samples.  Let's start by creating a new batch with a single vector.

In [12]:
storage_batch = schema.generate_empty_storage_batch(1)
storage_batch.shape

(1, 152)

Now let's say some baseline parameters (e.g. pulled from ResStock)

In [13]:
schema.update_storage_batch(storage_batch, parameter="FacadeRValue", value=20)
schema.update_storage_batch(storage_batch, parameter="RoofRValue", value=30)
schema.update_storage_batch(storage_batch, parameter="LightingPowerDensity", value=7.2)
schema.update_storage_batch(storage_batch, parameter="base_template", value=15)
storage_batch = np.concatenate([storage_batch for _ in range(4)], axis=0)
storage_batch

array([[ 0. ,  0. , 15. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  7.2,  0. ,  0. , 20. , 30. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. , 

Now let's set the orientations:

In [14]:
values = np.arange(4).reshape(-1,1)
parameter = "orientation"
schema.update_storage_batch(storage_batch, parameter=parameter, value=values)
schema[parameter].extract_storage_values_batch(storage_batch)

array([[0.],
       [1.],
       [2.],
       [3.]])

Looks good!  Now let's stack this up and begin generating some geometric variations.

In [15]:
orientations_per_base = 4
geometric_variations_per_orientation = 5

In [16]:
storage_batch = np.repeat(storage_batch, geometric_variations_per_orientation, axis=0)
storage_batch.shape

(20, 152)

In [17]:
schema["orientation"].extract_storage_values_batch(storage_batch)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.]])

Looks good!  let's start populating this: if we wanted to use repeating values, we could do nested loops:

In [18]:
n = geometric_variations_per_orientation # how many design vectors in this mini batch
for i in range(orientations_per_base):
	start = i*n # where this mini batch starts in the parent batch
	end = start + n # where this mini batch ends in the parent batch
	for j,parameter in enumerate(schema.parameters):
		if isinstance(parameter, ShoeboxGeometryParameter):
			name = parameter.name
			shape = parameter.shape_storage
			np.random.seed(j+20923) # arbitrary but reliable seed
			values = np.random.rand(n, *shape) 
			values = parameter.unnormalize(values)
			schema.update_storage_batch(storage_batch, index=(start,end), parameter=name, value=values)


In [19]:
schema["wwr_e"].extract_storage_values_batch(storage_batch)

array([[0.43423096],
       [0.77860127],
       [0.96052363],
       [0.43193474],
       [0.79141046],
       [0.43423096],
       [0.77860127],
       [0.96052363],
       [0.43193474],
       [0.79141046],
       [0.43423096],
       [0.77860127],
       [0.96052363],
       [0.43193474],
       [0.79141046],
       [0.43423096],
       [0.77860127],
       [0.96052363],
       [0.43193474],
       [0.79141046]])

In [20]:
schema["width"].extract_storage_values_batch(storage_batch)

array([[2.26897157],
       [4.37366836],
       [1.66250093],
       [2.66367318],
       [4.95818986],
       [2.26897157],
       [4.37366836],
       [1.66250093],
       [2.66367318],
       [4.95818986],
       [2.26897157],
       [4.37366836],
       [1.66250093],
       [2.66367318],
       [4.95818986],
       [2.26897157],
       [4.37366836],
       [1.66250093],
       [2.66367318],
       [4.95818986]])

Great, these are repeating correctly!  Now, suppose we want to just slightly perturb all of these so that they aren't perfectly repeating, but are close to repeating:

In [21]:
n = storage_batch.shape[0]
for i,parameter in enumerate(schema.parameters):
	if isinstance(parameter, ShoeboxGeometryParameter):
		name = parameter.name
		shape = parameter.shape_storage
		perturbations = np.random.rand(n,*shape)*0.2 - 0.1
		values = parameter.extract_storage_values_batch(storage_batch)
		values += perturbations
		schema.update_storage_batch(storage_batch,parameter=name,value=values)

schema["width"].extract_storage_values_batch(storage_batch)

array([[2.35024646],
       [4.40730885],
       [1.62272824],
       [2.6785684 ],
       [4.9320105 ],
       [2.29098495],
       [4.29247051],
       [1.74526617],
       [2.65644796],
       [5.04938303],
       [2.25792396],
       [4.35490027],
       [1.75458367],
       [2.71324522],
       [5.01682576],
       [2.20010338],
       [4.34929446],
       [1.59836104],
       [2.5765532 ],
       [5.03959253]])

Great!  We see that they are close to their previous values, but not identical.  

Alternatively, we might prefer to simply use fully random geometric variations for all of our orientation duplicates, rather than repeating the geometry across orientations:

In [22]:
n = storage_batch.shape[0]
for i,parameter in enumerate(schema.parameters):
	if isinstance(parameter, ShoeboxGeometryParameter):
		name = parameter.name
		shape = parameter.shape_storage
		values = np.random.rand(n,*shape)
		values = parameter.unnormalize(values)
		schema.update_storage_batch(storage_batch,parameter=name,value=values)

schema["width"].extract_storage_values_batch(storage_batch)

array([[3.51231112],
       [1.96563661],
       [4.45504452],
       [2.92512066],
       [3.32828155],
       [2.52865285],
       [4.03686039],
       [3.00049402],
       [1.8619203 ],
       [4.72915503],
       [4.85154758],
       [2.034229  ],
       [4.55501067],
       [4.31147279],
       [4.28423789],
       [2.25854047],
       [3.32435826],
       [4.44971593],
       [4.04077727],
       [3.20571423]])

Suppose this was our finished batch.  We can save it to an HDF5 file.  Let's say this was building 23 from our ResStock database.

In [23]:
import h5py
from storage import upload_to_bucket

In [24]:
# Update the building IDs
batch_id = 23 # suppose this is the base building we are drawing from
n = storage_batch.shape[0]
variation_ids = np.arange(n)
schema.update_storage_batch(storage_batch,parameter="batch_id",value=batch_id)
schema.update_storage_batch(storage_batch,parameter="variation_id",value=variation_ids)

# Write to an HDF5 file
slug = f"batch_{batch_id:04d}.hdf5"
outfile = f"./data/{slug}"
with h5py.File(outfile,"w") as f:
    f.create_dataset(name="storage_vectors", shape=storage_batch.shape, dtype=storage_batch.dtype, data=storage_batch)

# upload to cloud bucket for easy backup
destination = f"demo-batch-data/{slug}"
upload_to_bucket(destination, outfile)


## Simulation

Now let's suppose you want to simulate a design vector.  Let's open up an HDF5 file and read in only the first storage vector to get started.

In [27]:
batch_id = 23
slug = f"batch_{batch_id:04d}.hdf5"
outfile = f"./data/{slug}"
storage_vector = None
with h5py.File(outfile,'r') as f:
    storage_vector = f["storage_vectors"][0]

schema["batch_id"].extract_storage_values(storage_vector), schema["variation_id"].extract_storage_values(storage_vector)

(23.0, 0.0)

Now let's create a simulation object for this storage vector:

In [29]:
whitebox_sim = schema.create_whitebox_simulation(storage_vector)

Now we can take a look at the semantic objects that have been configured!

In [30]:
print(whitebox_sim.shoebox_config.facade_2_footprint)
print(whitebox_sim.shoebox_config.width)

0.7368189481003298
